In [ ]:
# standard imports
import sys
import numpy as np
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# import geometry classes
from geometry.node import node
from geometry.cell import cell
from geometry.mesh import rot_sym_mesh

# problem defining operators
from problem.isentropic_navier_stokes import convection, diffusion, skalar_product, curl

# projection based functions
from utilities.projection import isentropic_projection, isentropic_control_projection

# utility functions
from utilities.utilities import *
from utilities.pod import get_POD, get_activations

In [ ]:
# importing control library
from casadi import *

# reloading modules

modName = "utilities.projection"
del sys.modules[modName]
from utilities.projection import isentropic_projection, isentropic_control_projection

# Setting Custom Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./mesh_files/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class
mesh = rot_sym_mesh(meshio_mesh)

# Parameter List

In [ ]:
# PROBLEM CONSTANTS
n = mesh.n                  # number of nodes
N = mesh.N                  # number of cells
T = 220                     # number of snapshots
nu = 1.716e-5               # kinematic viscosity
mach = 0.6                  # mach number
sos = 340.29                # isentropic speed of sound
uInfty = mach * sos         # freestream velocity
Re = 100                    # Reynolds number of freeflow
dt = 0.0001                 # simulation time step
subsampling = 6             # subsampling for data
nondim = False              # nondimensionalization of results
chirp = True                # chirp dataset

# POD PARAMETERS
maxPOD = 12                 # maximum number of POD modes
gamma = 1.4                 # isentropic coefficient -> material property
alpha = 1/gamma             # inner product coefficient [(gamma - 1)/2 -> standard, 1 -> stagnation enthalpy]
include_avg = False         # include average flow in POD computation

# GALERKIN PARAMETERS
stabilization = "viscosity" # stabilization method for galerkin system
epsilon = 1e-4              # weight for sobolev norm stabilization -> experimental
if mach == 0.6:
    dt_ref = 0.0001 * subsampling     # snapshot equaivalent time step
    dt = 0.0005                       # galerkin time step
    shed_period = 0.0327              # hypothesized shed period
elif mach == 0.1:
    dt_ref = 0.001 * subsampling
    dt = 0.001
    shed_period = 0.18
elif mach == 0.01:
    dt_ref = 0.02 * subsampling
    dt = 0.02
    shed_period = 1.84
t0 = 0
tmax = shed_period * 10
temporalSpan = np.linspace(t0,tmax,int(tmax/dt)+1)

# OPTIMIZATION PARAMETERS
designDim = 8                       # number of design parameters
designVars = np.ones(designDim)     # design variables for gamma
control0 = 0                        # initial control
gamma_max = 10                      # maximum allowable control
phi0 = 0                            # costate end condition
epsilon = 1e-4                      # discontinuation criteria

In [ ]:
# reading file data
dataDir  = "./flow_data/lam_cyl_comp_air/mach 0.6 chirp/"
dataDict = read_csv_data(dataDir,delay=0,subsampling=subsampling,max_size=T)

# geometric information
x = dataDict["x"][:,0] - 0.5
y = dataDict["y"][:,0]

# fluid velocity and mach number
u = np.divide(dataDict['Momentum_x'], dataDict['Density'])
v = np.divide(dataDict['Momentum_y'], dataDict['Density'])
a = dataDict['Mach']

# construct state based data
q = np.vstack([u,v,a])
T = q.shape[1]
qAvg = np.mean(q,1)
qFluc = q - np.repeat(np.expand_dims(qAvg,1),T,axis=1)

# data for pod decomposition
if include_avg:
    podData = q
else:
    podData = qFluc

In [ ]:
### set scalar weights for multiplicative computation
# volume participations
volumeWeight = mesh.volume_weights
# additional weight for speed of sound
machWeight = (2 * alpha / (gamma - 1))
# construction of weights
innerWeight = np.concatenate([np.ones(n),np.ones(n),machWeight*np.ones(n)]) * volumeWeight

In [ ]:
def skalar_product(q1,q2,weights=innerWeight):
    return np.sum(q1*q2*weights)

# POD Decomposition

In [ ]:
[podModes, S] = get_POD(podData,skalar_product,20)
coeffs = get_activations(podData,podModes,skalar_product,20)
if nondim:
    coeffs /= uInfty

In [ ]:
# energy reconstruction threshold
epsilon = 0.99
acc = 0
dim = 0
while acc < epsilon:
    dim+=1
    acc = sum(S[:dim])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(dim))
if dim % 2 == 1:
    dim+=1

# manual value assignement
dim = 2

In [ ]:
# plot_activations(coeffs,dim,dt=dt*subsampling)

# Control Data Input

In [ ]:
controlDir  = "./flow_data/lam_cyl_comp_air/potVortex_0.6_1/"
controlDict = read_csv_data(controlDir,0,1,1)

uCon = np.divide(controlDict['Momentum_x'], controlDict['Density'])
vCon = np.divide(controlDict['Momentum_y'], controlDict['Density'])
aCon = controlDict['Mach']

# construct state based data
qCon = np.vstack([uCon,vCon,aCon]).squeeze()

# Galerkin Coefficients

In [ ]:
[b1, b2, L1, L2, Q] = isentropic_projection(mesh,podModes[:,:dim],qAvg,skalar_product)

In [ ]:
(nu * L1 + L2)[0]

# Control Function Method
Additional coefficients in Galerkin system

In [ ]:
[d1,d2,f,g,h] = isentropic_control_projection(mesh,podModes[:,:dim],qAvg,qCon,skalar_product)

# Galerkin System

In [ ]:
# initial conditions
a0 = coeffs[:dim,0]

# dynamic viscosity for stabilization
if stabilization=="viscosity":
    if mach == 0.6:
        # tuned
        if dim == 10:
            nu = 4.09 if chirp==False else 2.29 # 10 modes
        elif dim == 8:
            nu = 3.24 if chirp==False else 2.1  # 8 modes
        elif dim == 6:
            nu = 3.07 if chirp==False else 1.9  # 6 modes
        elif dim == 4:
            nu = 3.625 if chirp==False else 2.45# 4 modes
        elif dim == 2:  
            nu = 4.36  if chirp==False else 3.01# 2 modes #3.05 for LSODA

    elif mach == 0.1:
        # tuned
        if dim == 10:
            nu = 0.745   # 10 modes
        elif dim == 8:
            nu = 0.7445  # 8 modes
        elif dim == 6:
            nu = 0.7390  # 6 modes
        elif dim == 4:
            nu = 0.7635  # 4 modes
        elif dim == 2:  
            nu = 0.8625  # 2 modes

    elif mach == 0.01:
        # tuned
        if dim == 10:
            nu = 0.1535  # 10 modes
        elif dim == 8:
            nu = 0.151   # 8 modes
        elif dim == 6:
            nu = 0.1515  # 6 modes
        elif dim == 4:
            nu = 0.1515  # 4 modes
        elif dim == 2:  
            nu = 0.1550  # 2 modes

In [ ]:
# defining ode function for node based coefficients
def galerkin_system(t,a):
    # set global variables references
    global Q, L1, L2, b1, b2, nu
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu * L1[k,:] + L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q[k]),np.expand_dims(a,1))) 
    return a_dot

In [ ]:
sol = solve_ivp(galerkin_system,(t0,tmax),a0, method='RK45',t_eval=temporalSpan)
sol.message

In [ ]:
for i in range(2):
    plt.plot(sol.y[i,:])

# Equilibrium point -> Newton method

In [ ]:
def jacobianGalerkinSystem(x,L,Q):
    # analytical jacobian for Galerkin system
    dim = len(x)
    jacobian = np.zeros((dim,dim))
    for k in range(dim):
        for j in range(dim):
            jacobian[k,j] = L[k,j] + 2 * Q[k,j,j] * x[k]
            for i in range(dim):
                jacobian[k,j] += (Q[k,i,j] + Q[k,j,i]) * x[i]
    # return 
    return jacobian

equi = np.ones(dim)
fx = galerkin_system(0,equi)
L = (nu * L1 + L2)
while not np.isclose(fx,0).all():
    # jacobian at point
    jac = jacobianGalerkinSystem(equi,L,Q)
    # solving linear system for equilibirum update
    equi = np.linalg.solve(jac, - fx) + equi
    # solve for function value
    fx = galerkin_system(0,equi)

print("Equilibrium: " + str(equi))

# Stability

In [ ]:
A = jacobianGalerkinSystem(equi,L,Q) # linearized matrix system
w, _ = np.linalg.eig(A)

# checking for stability property
if (np.real(w) < 0).all():
    print("System is stable around equilibrium!")
elif (np.real(w) > 0).any():
    print("System is unstable around equilibrium!")
else:
    print("No conclusions about stabilty from linearization!")

Note that the stabilization using artifical viscosity presents a combination of saddle-node and trasncritical bifurcation. 

# Comparison of Limit Cycles

In [ ]:
# Galerkin model 
d = int(dim/2)
fig,ax = plt.subplots(d,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = temporalSpan 
phase_shift = 54
plot_time_span = 100 * dt
orig_number = int(plot_time_span / (dt_ref)) #  *1.12))
t_orig = np.linspace(t[-100],tmax-dt_ref,num=orig_number)

# add plots over full time domain
for i in range(dim):
    if dim <= 2:
        ax[int(i%2)].plot(t[-100:],sol.y[i,-100:]/(sos * mach))
        ax[int(i%2)].plot(t_orig,coeffs[i,-(orig_number + phase_shift):-phase_shift]/(sos * mach))
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t[-100:],sol.y[i,-100:]/(sos * mach))
        ax[int(i/2)][int(i%2)].plot(t_orig,coeffs[i,-(orig_number + phase_shift):-phase_shift]/(sos * mach))
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))

# Optimal Control

In [ ]:
# control system with casadi
designDim = 10
tmax = np.ceil((10 * shed_period) / dt) * dt
Nt = int(tmax/dt) + 1
uMax = 2
x = MX.sym('x', dim)
p = MX.sym('p')
dp = MX.sym('dp')

# expression for ode rhs 
tmp = [0 for i in range(dim)]
for k in range(dim):
    tmp[k] = nu * b1[k] + b2[k] + p * (nu * d1[k] + d2[k]) + p * p * f[k] #+ h[k] * dp
    for i in range(dim):
        tmp[k] += (nu * L1[k,i] + L2[k,i]) * x[i] + p * g[k,i] * x[i]
        for j in range(dim):
            tmp[k] += Q[k,i,j] * x[i] * x[j]    
rhs = vertcat(tmp[0],tmp[1])

In [ ]:
rhs = vertcat([tmp[i] for i in range(dim)])

In [ ]:
# ode declaration
ode = {'x':x,'p':p,'ode':rhs}

# integrator construction
F = integrator('F','cvodes',ode,{'tf':dt})

# control vector
u = MX.sym('u', Nt, 1)

# initial conditions
x = coeffs[:dim,0]
for n in range(Nt):
    res = F(x0=x,p=u[n])
    x = res['xf']

# nonlinear program declaration
nlp = {'x':u,'f':dot(x,x)+2.71**(u**2-uMax**2),'g':x}

# solve using IPOPT
solver = nlpsol('solver','ipopt',nlp)
res = solver(x0=x0,lbg=0,ubg=0)

plot(res["x"])